In [0]:
from Functions import *

In [0]:
# spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

In [0]:
#Beginning of the program
load_control_table_entries = fetch_data_flows(spark).collect()

for entry in load_control_table_entries:
    tables = intialize_tables(spark, entry)
    
    if entry['sourceFormat'] == "Oracle DB":
        url, jdbc_driver_value, username, password = get_jdbc_credentials(entry)
        for table in tables:
            print("Started table " + table['targetTablename'], datetime.now())
            if table_dependency_check(spark, table['loadDependency']):
                if table['tableProperties']['tablesize'] == "small":
                    load_data(spark, table, url, jdbc_driver_value, username, password)
                
                elif table['tableProperties']['tablesize'] == "huge":
                    if table['splitLoad'] == "N":
                        load_data(spark, table, url, jdbc_driver_value, username, password)
                    elif table['splitLoad'] == "Y":
                        recurrence = table['splitLoadConfigurations']['recurrence']  #monthly, quarterly, yearly
                        cdc_column = table['splitLoadConfigurations']['splitColumn']
                        split_load(spark, table, cdc_column, url, jdbc_driver_value, username, password, recurrence)
                update_ingestion_entities_status(spark, "Completed", None, table['JobId'])
            else:
                print("Dependent tables are not completed")
    update_dataflow_status(spark, entry['dataFlowGroup'], "Ready for DLT")


In [0]:
# %run ./Functions/Functions

In [0]:
# #Beginning of the program
# load_control_table_entries = fetch_data_flows().collect()
# # load_control_table_entries = get_load_control_table_entries().collect()

# for entry in load_control_table_entries:
#     tables = intialize_tables(entry)
    
#     if entry['sourceFormat'] == "Oracle DB":
#         url, jdbc_driver_value, username, password = get_jdbc_credentials(entry)
#         for table in tables:
#             print("Started table " + table['targetTablename'], datetime.now())
#             if table_dependency_check(table['loadDependency']):
#                 if table['tableProperties']['tablesize'] == "small":
#                     load_data(table, url, jdbc_driver_value, username, password)
                
#                 elif table['tableProperties']['tablesize'] == "huge":
#                     if table['splitLoad'] == "N":
#                         load_data(table, url, jdbc_driver_value, username, password)
#                     elif table['splitLoad'] == "Y":
#                         recurrence = table['splitLoadConfigurations']['recurrence']  #monthly, quarterly, yearly
#                         cdc_column = table['splitLoadConfigurations']['splitColumn']
#                         split_load(table, cdc_column, url, jdbc_driver_value, username, password, recurrence)
#                 update_ingestion_entities_status("Completed", None, table['JobId'])
#             else:
#                 print("Dependent tables are not completed")
#     update_dataflow_status(entry['dataFlowGroup'], "Ready for DLT")
